In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import nest
import numpy as np
import os
import uuid
from pynestml.frontend.pynestml_frontend import to_nest, install_nest

NEST_SIMULATOR_INSTALL_LOCATION = "/home/beeblebrox/nest-simulator-build"   # XXX: CHANGE ME!


# NESTML Izhikevich tutorial

Introduction
------------

The aim of this exercise is to obtain familiarity with NESTML by completing a partial model of the Izhikevich neuron [1]_.


Prerequisites
-------------

You need to have a working NEST Simulator and NESTML installation, see [Installing NESTML](https://nestml.readthedocs.io/en/latest/installation.html).

You need to be able to run NESTML to generate and build model code. The recommended way is via the Python API. See [Running NESTML](https://nestml.readthedocs.io/en/latest/running.html).

### Paths

We assume here that NEST Simulator is installed to `/home/beeblebrox/nest-simulator-build` and that we will generate code in `/tmp/nestml-component`.

When starting the Python interpreter, these paths can be passed as environment variables, for example as follows (note the Python 3.6 version number, adapt as needed):

```bash
PYTHONPATH=$PYTHONPATH:/home/beeblebrox/nest-simulator-build/lib/python3.6/site-packages LD_LIBRARY_PATH=/tmp/nestml-component ipython3
```

Note that on MacOS, `LD_LIBRARY_PATH` is called `DYLD_LIBRARY_PATH`.


The Izhikevich model
--------------------

A simple model for spiking neurons that nevertheless can exhibit a wide variety of dynamical behaviour, depending on its parameter values [1]_. It is defined as follows:

\begin{align}
\frac{dv}{dt} &= 0.04 v^2 + 5 v + 140 - u + I\\
\frac{du}{dt} &= a (b v - u)
\end{align}

State update:

\begin{align}
   &\text{if}\;\; v \geq V_{th}:\\
   &\;\;\;\; v \text{ is set to } c\\
   &\;\;\;\; u \text{ is incremented by } d\\
   & \, \\
   &v \text{ jumps on each spike arrival by the weight of the spike}
\end{align}

Example parameters for regular spiking: 

\begin{align}
a&=0.02\\
b&=0.2\\
c&=-65\\
d&=8
\end{align}


Task 1: Finish the model
------------------------

In the file :ref:`izhikevich_task.nestml <izhikevich_task.nestml>`, only a subset of the parameters, state equations and update block is implemented.

Open the .nestml file in a text editor and finish the partially-completed model.

For reference, the solution is included as `izhikevich_solution.nestml`.

### NESTML code generation

Assume that our NESTML input model is at `/home/beeblebrox/nestml-tutorial/izhikevich_solution.nestml`. To generate code and build a dynamic library that can be loaded as a user module in NEST Simulator:

In [ ]:
from pynestml.frontend.pynestml_frontend import to_nest, install_nest
to_nest(input_path="/home/beeblebrox/nestml-tutorial/izhikevich_solution.nestml", target_path="/tmp/nestml-component", logging_level="INFO")
install_nest("/tmp/nestml-component", "/home/beeblebrox/nest-simulator-build")

Check the generated log output for any potential error messages or warnings.

Task 2: Parameter space exploration
-----------------------------------

Using the PyNEST API, the model can be instantiated and simulated in NEST.

Perform a parameter space exploration to reproduce the bottom eight panels from [1]_, figure 2.


### Instantiate model in NEST Simulator and run

The following code will load the generated NESTML module containing the model (`nest.Install("nestmlmodule")`), perform the instantiation of the model (`nest.Create("izhikevich_tutorial")`), inject a constant current and run the simulation for 250 ms.

In [ ]:
nest.set_verbosity("M_WARNING")
nest.ResetKernel()
nest.Install("nestmlmodule")

neuron = nest.Create("izhikevich_tutorial")
voltmeter = nest.Create("voltmeter")

voltmeter.set({"record_from": ["v"]})
nest.Connect(voltmeter, neuron)

cgs = nest.Create('dc_generator')
cgs.set({"amplitude": 25.})
nest.Connect(cgs, neuron)

nest.Simulate(250.)

plt.plot(voltmeter.get("events")["times"], voltmeter.get("events")["v"])
plt.show()

## References

Eugene M. Izhikevich, "Simple Model of Spiking Neurons", IEEE Transactions on Neural Networks, Vol. 14, No. 6, November 2003

## Copyright

This file is part of NEST.

Copyright (C) 2004 The NEST Initiative

NEST is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 2 of the License, or (at your option) any later version.

NEST is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with NEST.  If not, see <http://www.gnu.org/licenses/>.
